## Carregando arquivo de dados climáticos

Na tabela existem alguns valores -9999 que são erros de leitura da estação meteorológica que vamos ter que excluir. Aqui transformei estes valores em None/NaN (Not a Number)

In [1]:
import pandas as pd
clima = pd.read_csv (r'Entrada\Clima_2012_2021.csv', decimal=",", sep=";")
clima.replace(-9999.0, None, inplace=True)

Ajusta nomes das colunas, e também ajusta o formato da coluna de data, convertendo de string para date. Também ordena a tabela pela data.

In [2]:
clima.columns=['DATA','HORA','PRECIPITACAO','PRESSAO ATMOSFERICA','RADIACAO GLOBAL','TEMPERATURA','PONTO DE ORVALHO','UMIDADE RELATIVA DO AR','VELOCIDADE DO VENTO']
clima["DATA"] = pd.to_datetime(clima["DATA"],format="%d/%m/%Y").dt.date
clima = clima.sort_values(by="DATA")

Agrupa o valor de cada variável por dia, por exemplo, temperatura média do DIA 01/02/2015. Isso porque os valores estão por hora, precisamos deles por dia para fazer as análises já que os focos de incêndios são diários.

Também ajusta as colunas e indices da nova tabela.

In [3]:
agrupados=clima.groupby('DATA').agg({
    'PRECIPITACAO':['sum','mean'],
    'PRESSAO ATMOSFERICA':'mean',
    'RADIACAO GLOBAL':'mean',
    'TEMPERATURA':['min','max','mean'],
    'PONTO DE ORVALHO':'mean',
    'UMIDADE RELATIVA DO AR':['min','mean'],
    'VELOCIDADE DO VENTO':'mean'
})

agrupados.columns=['PRECIPITACAO TOTAL','PRECIPITACAO MEDIA','PRESSAO ATMOSFERICA MEDIA','RADIACAO GLOBAL MEDIA','TEMPERATURA MINIMA',
'TEMPERATURA MAXIMA','TEMPERATURA MEDIA','PONTO DE ORVALHO MEDIA','UMIDADE RELATIVA DO AR MINIMA','UMIDADE RELATIVA DO AR MEDIA','VELOCIDADE DO VENTO MEDIA']
agrupados=agrupados.reset_index()

## Carrega arquivo de Focos de incêndio

Carrega arquivo de focos de incêndios: cada entrada é um foco de incêndio, contendo a data e hora, e outros dados irrelevantes para nossas análises.

Também realiza o ajuste no formato da coluna de data, convertendo de string para date. Também ordena a tabela pela data.

In [4]:
focos = pd.read_csv(r'Entrada\Focos_2012_2021.CSV', decimal=",", sep=";")
focos["datahora"] = pd.to_datetime(focos["datahora"]).dt.date

focos=focos.groupby("datahora").size().sort_values().reset_index("datahora")
focos.columns = ["DATA","QUANTIDADE FOCOS"]

## Finalizando
Junta as duas tabelas em uma única (clima + focos), utilizando a data como chave e concatenando a coluna de quantidade de focos no final

In [5]:
merjado = pd.merge(agrupados, focos, on="DATA", how="left")

Troca os valores NaN por 0, e cria uma coluna extra, indicando se houve (True) ou não (False) algum foco de incêndio naquela data, e salva como um novo csv.

In [6]:
import numpy as np
merjado["QUANTIDADE FOCOS"] = merjado["QUANTIDADE FOCOS"].fillna(0)
merjado["OCORREU FOGO"] = np.where(merjado['QUANTIDADE FOCOS'] > 0, True, False)
merjado.to_csv(r'Saida\dados_finais.csv', index=False)